![logo](https://joss.theoj.org/logo_large.jpg)

# Intro
So, I came across a paper called [Riroriro: Simulating gravitational waves and evaluatingtheir detectability in Python](https://joss.theoj.org/papers/10.21105/joss.02968) which describes a Python package for generating and analyzing simulated gravitational waves. The data set we have in this competition is dominated by detector noise. By exploring noise-free simulated waveforms we can gain a much better understanding of the properties of the signals. The host probably used a very similar (or the same?) numerical library to generate the gravitational waves in the dataset.  

First, install Riroriro:

In [ ]:
!pip install riroriro

The source code can be found on [GitHub](https://github.com/wvanzeist/riroriro).

In [ ]:
import numpy as np
import riroriro.inspiralfuns as ins
import riroriro.mergerfirstfuns as me1
import riroriro.matchingfuns as mat
import riroriro.mergersecondfuns as me2
import librosa
import librosa.display
import math
import matplotlib.pyplot as plt

# Generate gravitational waves
Riroriro comes with a [nice tutorial](https://github.com/wvanzeist/riroriro_tutorials/blob/main/example_GW.ipynb) for how to generate GW signals, and below we simply copy all that code (except for size reduction) into a function:

In [ ]:
# Parameters:
# logMC: system mass (0.0-2.0)
# q: mass ratio (0.1-1.0)
# D: distance (Mpc)
# merger_type: 'BH'=binary black hole merger, 'NS'=binary neutron star merger
# flow: low frequency (Hz) 
def gen_gw(logMc=1.4, q=0.8, D=100.0, flow=10.0, merger_type='BH'):
    M, eta = ins.get_M_and_eta(logMc=logMc,q=q)
    start_x = ins.startx(M,flow)
    end_x = ins.endx(eta,merger_type)
    x, xtimes, dt = ins.PN_parameter_integration(start_x,end_x,M,eta)
    realtimes = ins.inspiral_time_conversion(xtimes,M)
    i_phase, omega, freq = ins.inspiral_phase_freq_integration(x,dt,M)
    r, rdot = ins.radius_calculation(x,M,eta)
    A1, A2 = ins.a1_a2_calculation(r,rdot,omega,D,M,eta)
    i_Aorth, i_Adiag = ins.inspiral_strain_polarisations(A1,A2,i_phase)
    i_amp = ins.inspiral_strain_amplitude(i_Aorth,i_Adiag)
    i_time = realtimes
    i_omega = omega
    sfin, wqnm = me1.quasi_normal_modes(eta)
    alpha, b, C, kappa = me1.gIRS_coefficients(eta,sfin)
    fhat, m_omega = me1.merger_freq_calculation(wqnm,b,C,kappa)
    fhatdot = me1.fhat_differentiation(fhat)
    m_time = me1.merger_time_conversion(M)
    min_switch_ind = mat.min_switch_ind_finder(i_time,i_omega,m_time,m_omega)
    final_i_index = mat.final_i_index_finder(min_switch_ind,i_omega,m_omega)
    time_offset = mat.time_offset_finder(min_switch_ind,final_i_index,i_time,m_time)
    i_m_time, i_m_omega = mat.time_frequency_stitching(min_switch_ind,final_i_index,time_offset,i_time,i_omega,m_time,m_omega)
    i_m_freq = mat.frequency_SI_units(i_m_omega,M)
    m_phase = me2.merger_phase_calculation(min_switch_ind,final_i_index,i_phase,m_omega)
    i_m_phase = me2.phase_stitching(final_i_index,i_phase,m_phase)
    m_amp = me2.merger_strain_amplitude(min_switch_ind,final_i_index,alpha,i_amp,m_omega,fhat,fhatdot)
    i_m_amp = me2.amplitude_stitching(final_i_index,i_amp,m_amp)
    m_Aorth, m_Adiag = me2.merger_polarisations(final_i_index,m_amp,m_phase,i_Aorth)
    i_m_Aorth, i_m_Adiag = me2.polarisation_stitching(final_i_index,i_Aorth,i_Adiag,m_Aorth,m_Adiag)
    return np.array(i_m_time), np.array(i_m_Aorth), np.array(i_m_Adiag), np.array(i_m_freq)

The function returns two waves that represent orthogonal/diagonal waves. The output timescale that is returned is non-linear, so to convert these signals into uniform sampled signals as in the dataset, we need to resample. The function below will resample the gravitational wave signals to 2048Hz. It is crude though, based on nearest sample, but good enough for studying spectrums. Interpolation would be more proper.

In [ ]:
SR = 2048 # target sample rate (Hz)
# Parameters:
# dt: time series
# amp: amplitude signal
# seg: output sequence length (seconds)
def resample(dt, amp, seg=2.0):
    end = dt[-1]
    start = end - seg
    d = np.zeros(int(SR*seg))
    for i in range((int(SR*seg))):
        t = start + i/SR
        d[i] = amp[np.where(dt == dt[np.abs(dt-t).argmin()])[0][0]]
    return d

Another helper function for plotting the last part of the GW signal (containing the chirp):

In [ ]:
def plot_sig(dt, sig1, sig2=None, seg=2.0):
    end = dt[-1]
    start = end - seg
    plt.figure(1)
    plt.plot(dt, sig1)
    peak = np.max(np.abs(sig1))
    plt.axis([start,end,np.min(sig1)-peak/10,np.max(sig1)+peak/10])
    if sig2 is not None:
        plt.plot(dt, sig2)
    plt.xlabel('Time (s)')
    plt.ylabel('Strain amplitude')

# Test the signal generation
Now, let's test the code by emulating the first GW detected (GW150914).

In [ ]:
m_time, m_Aorth, m_Adiag, m_freq = gen_gw(logMc=1.4, q=0.2)

Plot the signals, first the last two seconds, then the last 0.1s (ringdown part).

In [ ]:
fig = plt.figure(figsize=(16,8))
plt.subplot(2, 1, 1)
plot_sig(m_time, m_Aorth, m_Adiag, seg=2)
plt.subplot(2, 1, 2)
plot_sig(m_time, m_Aorth, m_Adiag, seg=.1)

Next, resample the signal to 2048Hz (only the orthogonal part for simplicity):

In [ ]:
d1 = resample(m_time, m_Aorth, 2.0)

Compare with the original:

In [ ]:
fig = plt.figure(figsize=(16,16))
plt.subplot(2, 2, 1)
plot_sig(m_time, m_Aorth, seg=2)
plt.title('Original')
plt.subplot(2, 2, 2)
plot_sig(m_time, m_Aorth, seg=.1)
plt.title('Original (zoomed)')
plt.subplot(2, 2, 3)
plt.plot(d1)
plt.title('Resampled to 2048Hz')
plt.subplot(2, 2, 4)
plt.plot(d1[-205:])
plt.title('Resampled to 2048Hz (zoomed)');

Yes, we are happy with that! The simulated wave data also has a frequency vector. Let's visualize that:

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))
plt.plot(m_time, m_freq, label="Min: {}Hz, Max: {}Hz".format(int(np.min(m_freq)), int(np.max(m_freq))))
peak = np.max(np.abs(m_freq))
plt.axis([m_time[-1] - 2.0 if m_time[-1] >= 2.0 else m_time[0], m_time[-1], 0 , np.max(m_freq)+peak/10])
ax.legend()
plt.xlabel('Time (s)')
plt.ylabel('Frequency (Hz)');

So the gravitational wave signal starts around 10Hz and rises rapidly to 181Hz at the end. The start frequency is actually defined by the 'flow' parameter.

# Spectrum
We can now do a constant-Q transform (here using librosa) to visualize the now familiar chirp part of the gravitational wave signal.


In [ ]:
hop_length = 64
C = np.abs(librosa.cqt(d1/np.max(d1), sr=SR, hop_length=hop_length, fmin=8, filter_scale=0.8, bins_per_octave=12))
fig, ax = plt.subplots(figsize=(10,10))
img = librosa.display.specshow(librosa.amplitude_to_db(C, ref=np.max),
                               sr=SR*2, hop_length=hop_length, bins_per_octave=12, ax=ax)
ax.set_title('Constant-Q power spectrum');

## Chirp frequency vs. system mass
Now let's see how the system mass and mass ratio parameters effect the frequency content of the GW signal. We keep one parameter constant (middle value) while varying the other. Starting with system mass (logMc parameter).

In [ ]:
hop_length = 64
mass = [0.1, 0.7, 2.0]
fig = plt.figure(figsize=(20,15))
q = [0.45]
for m in range(len(mass)):
    m_time, m_Aorth, _, m_freq = gen_gw(logMc=mass[m], q=q[0])
    rd = resample(m_time, m_Aorth, 2.0)
    # time series
    ax = plt.subplot(len(mass), 4, 1+m*4)
    plt.plot(rd)
    plt.title('Signal (logMc={}, q={})'.format(mass[m], q[0]))
    # zoomed times series (chirp)
    ax = plt.subplot(len(mass), 4, 2+m*4)
    plt.plot(rd[-205:])
    plt.title('Signal chirp (zoomed)')
    # frequency content (last 2s)
    ax = plt.subplot(len(mass), 4, 3+m*4)
    plt.plot(m_time, m_freq, label="Min: {}Hz, Max: {}Hz".format(int(np.min(m_freq)), int(np.max(m_freq))))
    peak = np.max(np.abs(m_freq))
    plt.axis([m_time[-1] - 2.0 if m_time[-1] >= 2.0 else m_time[0], m_time[-1], 0 , np.max(m_freq)+peak/10])
    ax.legend()
    plt.xlabel('Time (s)')
    plt.ylabel('Frequency (Hz)');
    plt.title('Frequency content (last 2s)')
    # Q-Transform
    ax = plt.subplot(len(mass), 4, 4+m*4)
    C = np.abs(librosa.cqt(rd/np.max(rd), sr=SR, hop_length=hop_length, fmin=8, filter_scale=0.8, bins_per_octave=12))
    img = librosa.display.specshow(librosa.amplitude_to_db(C, ref=np.max),
                                   sr=SR*2, hop_length=hop_length, bins_per_octave=12, ax=ax)
    ax.set_title('Constant-Q power spectrum');

## Chirp frequency vs. mass ratio
Next mass ratio (q parameter).

In [ ]:
hop_length = 64
mass = [1.0]
fig = plt.figure(figsize=(20,15))
q = [0.1, 0.45, 1.0]
for m in range(len(q)):
    m_time, m_Aorth, _, m_freq = gen_gw(logMc=mass[0], q=q[m])
    rd = resample(m_time, m_Aorth, 2.0)
    # time series
    ax = plt.subplot(len(q), 4, 1+m*4)
    plt.plot(rd)
    plt.title('Signal (logMc={}, q={})'.format(mass[0], q[m]))
    # zoomed times series (chirp)
    ax = plt.subplot(len(q), 4, 2+m*4)
    plt.plot(rd[-205:])
    plt.title('Signal chirp (zoomed)')
    # frequency content (last 2s)
    ax = plt.subplot(len(q), 4, 3+m*4)
    plt.plot(m_time, m_freq, label="Min: {}Hz, Max: {}Hz".format(int(np.min(m_freq)), int(np.max(m_freq))))
    peak = np.max(np.abs(m_freq))
    plt.axis([m_time[-1] - 2.0 if m_time[-1] >= 2.0 else m_time[0], m_time[-1], 0 , np.max(m_freq)+peak/10])
    ax.legend()
    plt.xlabel('Time (s)')
    plt.ylabel('Frequency (Hz)');
    plt.title('Frequency content (last 2s)')
    # Q-Transform
    ax = plt.subplot(len(q), 4, 4+m*4)
    C = np.abs(librosa.cqt(rd/np.max(rd), sr=SR, hop_length=hop_length, fmin=8, filter_scale=0.8, bins_per_octave=12))
    img = librosa.display.specshow(librosa.amplitude_to_db(C, ref=np.max),
                                   sr=SR*2, hop_length=hop_length, bins_per_octave=12, ax=ax)
    ax.set_title('Constant-Q power spectrum');

## Amplitude vs distance
Do gravitational waves follow the general Inverse-square law (double the distance and get 0.25 of the amplitude)?

In [ ]:
hop_length = 64

fig = plt.figure(figsize=(20,15))
dist = [100., 200., 400.]
for m in range(len(dist)):
    m_time, m_Aorth, _, m_freq = gen_gw(logMc=1.4, q=0.2, D=dist[m])
    rd = resample(m_time, m_Aorth, 2.0)
    # time series
    ax = plt.subplot(len(dist), 3, 1+m*3)
    plt.plot(rd)
    plt.title('Signal (D={} Mpc)'.format(int(dist[m])))
    # zoomed times series (chirp)
    ax = plt.subplot(len(dist), 3, 2+m*3)
    plt.plot(rd[-205:])
    plt.title('Signal chirp (zoomed)')
    # Q-Transform
    ax = plt.subplot(len(dist), 3, 3+m*3)
    if m == 0:
        smax = np.max(rd)
    C = np.abs(librosa.cqt(rd/smax, sr=SR, hop_length=hop_length, fmin=8, filter_scale=0.8, bins_per_octave=12))
    if m == 0:
        Cmax = np.max(C)
    img = librosa.display.specshow(librosa.amplitude_to_db(C, ref=Cmax), # was np.max
                                   sr=SR*2, hop_length=hop_length, bins_per_octave=12, ax=ax)
    ax.set_title('Constant-Q power spectrum');

Surprise - they do not! If the distance is doubled, we get 0.5 of the amplitude! Read more about this [here](https://www.forbes.com/sites/startswithabang/2019/03/02/ask-ethan-why-dont-gravitational-waves-get-weaker-like-the-gravitational-force-does/?sh=260758c72f58).

# Add noise to the signal
Next we want to see how the signal looks after adding detector noise. Half the training data is simulated detector noise without signal (target=0), so all we have to do is to add our generated signal to one of those numpy files... Let's pick the first file with target=0:

In [ ]:
noise = np.load('../input/g2net-gravitational-wave-detection/train/0/0/0/00001f4945.npy')
plt.plot(noise[1,:]);

Then we add the GW150914 signal we created above to the noise data with different signal to noise ratios, and check how well the constant-Q transform captures the chirp:

In [ ]:
gain = [0.0, 1.0, 1/2, 1/8, 1/16, 1/32]
fig = plt.figure(figsize=(15,25))

def add_noise(gain, sig):
    nd = noise[1,:] + gain*sig
    return nd

for i in range(len(gain)):
    nd = add_noise(gain[i], d1)
    # time signal
    ax = plt.subplot(len(gain), 2, 1+i*2)
    plt.plot(nd)
    if i == 0:
        plt.title('Noise only')
    else:
        plt.title('Noise + signal, gain={}'.format(gain[i]))
    # constant-Q transform
    ax = plt.subplot(len(gain), 2, 2+i*2)
    C = np.abs(librosa.cqt(nd/np.max(nd), sr=SR, hop_length=hop_length, fmin=8, filter_scale=0.8, bins_per_octave=12))
    img = librosa.display.specshow(librosa.amplitude_to_db(C, ref=np.max),
                                   sr=SR*2, hop_length=hop_length, bins_per_octave=12, ax=ax)
    if i == 0:
        ax.set_title('Constant-Q power spectrum')
    else:
        ax.set_title('Constant-Q power spectrum (signal @{}dB)'.format(int(10*math.log10(gain[i]))));

At -12dB the chirp is just barely visible!

# Signal position within the 2s time series
In the plots above, the GW signal ends at the very end of the 2s window. In the dataset the signals typically end somewhere in the last 0.5s part. This can be observed in Q-transforms of strong signals within the dataset. Below a few Q-Transforms are visualized with the signal ending at different times within the 2s window.

In [ ]:
fig = plt.figure(figsize=(12,20))
d2 = np.concatenate((d1, np.zeros(4096))) # pad with 2s of zeros
pos = [0.5, 0.7, 0.8, 0.9, 0.99]
hop_length = 64

for i in range(len(pos)):
    # time series
    ax = plt.subplot(len(pos), 2, 1+i*2)
    start = 4096 - int(4096*pos[i])
    plt.plot(d2[start:start+4096])
    plt.title('Signal ending at {}s'.format(pos[i]*2))
    # Q-transform
    ax = plt.subplot(len(pos), 2, 2+i*2)
    C = np.abs(librosa.cqt(d2[start:start+4096]/np.max(d2), sr=SR, hop_length=hop_length, fmin=8, filter_scale=0.8, bins_per_octave=12))
    img = librosa.display.specshow(librosa.amplitude_to_db(C, ref=np.max),
                                   sr=SR*2, hop_length=hop_length, bins_per_octave=12, ax=ax)
    ax.set_title('Constant-Q power spectrum')

# Create noise signals from PSD
So, let's say we have a power spectrum density of the detector noise - how can we generate random noise signals from that? The power spectral density is a representation of statistical power distribution across frequency range of a specific signal. The procedure is quite simple:  
  * Make sure the number of samples in your given PSD is M = N/2 + 1, where N is desired FFT size
  * Give each spectral component a random phase, uniformly distributed between 0 and 360 degrees (or 0 and 2Pi radians)
  * Multiply the PSD amplitudes with random numbers with variance 1
  * Perform inverse FFT to obtain the time series  
  
But first we need to obtain the PSD for the three detectors. There are some unofficial PSDs in the [riroriro tutorial project](https://github.com/wvanzeist/riroriro_tutorials). So we fetch them below. Also we install a package for spectral resampling called [SpectRes](https://github.com/ACCarnall/SpectRes).

In [ ]:
!git clone https://github.com/wvanzeist/riroriro_tutorials.git
!pip install spectres

Let's take a look at one of those PSDs (Livo Livingston):

In [ ]:
psd_liv = np.genfromtxt('riroriro_tutorials/noise_spectra/o3_h1.txt') #LIGO Livingston
plt.plot(psd_liv[:,0], np.log10(psd_liv[:,1]));

The frequency vector starts at 10Hz and ends at 5kHz. And frequencies are not uniform (are they ever in astrophysics?). So we need to resample to our desired frequency interval of 0-1024Hz. We create a function for all the steps mentioned above:

In [ ]:
from spectres import spectres

# convert polar coordinates to rectangular format
def P2R(A, phi):
    return A * (np.cos(phi) + np.sin(phi)*1j)

# input is a 2 dimensional power spectrum: frequencies and amplitudes
def rand_wave(power_spectrum):
    regrid = np.arange(0., 1024.5, .5) # note: 2049 length to get 4096 samples from irfft
    # resample spectrum
    PDS, _ = spectres(regrid, power_spectrum[:,0], power_spectrum[:,1],
                      spec_errs=np.zeros(len(power_spectrum[:,0])), fill=0., verbose=False)
    # add random phase and amplitude
    ph = np.random.uniform(0.,2*np.pi,len(PDS)) # random phase
    PDS *= np.random.randn(len(PDS)) # random amplitude
    Z=P2R(PDS, ph) # polar to rectangular format
    Z *= 100. # scale factor (to get about the right signal amplitude)
    return np.fft.irfft(Z) 

Then let's generate a few signals:

In [ ]:
fig = plt.figure(figsize=(20,20))
for m in range(16):
    ax = plt.subplot(4, 4, 1+m)
    plt.plot(rand_wave(psd_liv));

Looks familiar! But we do not know anything about how the host generated the noise in the data files, so this bit is maybe not very useful for improving scores...

# Summary

From the plots above, we can see that the bigger the system mass, the lower the chirp frequency (as expected). And for small system masses, the chirp frequencies can reach many kHz. The mass ratio has less effect on the chirp, but the closer the masses are the higher the frequency. We have also seen how to add noise from data files with target=0, or even by creating new noise data form a PSD.